<a href="https://colab.research.google.com/github/vandanavk19/Project/blob/main/MinorProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipython-autotime
%load_ext autotime

time: 2.53 ms (started: 2021-05-19 11:10:53 +00:00)


In [2]:
import os
import numpy as np

import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
 
target = []
images=[]
flat_data = []

datadir ='/content/drive/MyDrive/MinorProject/Images'

categories = ['Carrot','Spinach','Tomato']

for i in categories:
  class_num=categories.index(i)
  path = os.path.join(datadir,i)
  for img in os.listdir(path):
      img_array = imread(os.path.join(path,img))
      img_resized = resize(img_array,(150,150,3)) 
      flat_data.append(img_resized.flatten())
      images.append(img_resized)
      target.append(class_num)
flat_data = np.array(flat_data)
target = np.array(target)
images=np.array(images)
      

time: 48.9 s (started: 2021-05-19 11:10:53 +00:00)


In [3]:
# Splitting the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.3,random_state=100)

time: 206 ms (started: 2021-05-19 11:11:42 +00:00)


In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
param_grid = [{'C':[1,10,100,1000],'kernel':['linear']},
              {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},]
svc=svm.SVC(probability=True)
clf=GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

time: 2min 6s (started: 2021-05-19 11:11:42 +00:00)


In [5]:
y_pred=clf.predict(x_test)
y_pred

array([1, 2, 1, 0, 1, 1, 1, 1, 2, 0, 0, 1, 0, 2, 2, 2, 1, 0, 0, 2, 0, 0,
       0, 2, 1, 2, 0])

time: 199 ms (started: 2021-05-19 11:13:49 +00:00)


In [6]:
y_test

array([1, 2, 0, 0, 1, 1, 1, 1, 0, 2, 0, 1, 0, 2, 0, 2, 1, 2, 2, 2, 0, 0,
       0, 0, 1, 0, 0])

time: 4.05 ms (started: 2021-05-19 11:13:49 +00:00)


In [7]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

time: 1.75 ms (started: 2021-05-19 11:13:49 +00:00)


In [8]:
accuracy_score(y_pred,y_test)

0.7037037037037037

time: 5.56 ms (started: 2021-05-19 11:13:49 +00:00)


In [9]:
confusion_matrix(y_pred,y_test)

array([[7, 0, 3],
       [1, 8, 0],
       [4, 0, 4]])

time: 6.77 ms (started: 2021-05-19 11:13:49 +00:00)


In [10]:
import pickle
pickle.dump(clf,open('img_model.p','wb'))

time: 92.1 ms (started: 2021-05-19 11:13:49 +00:00)


In [11]:
model=pickle.load(open('img_model.p','rb'))

time: 26.2 ms (started: 2021-05-19 11:13:49 +00:00)


In [12]:
flat_data=[]
url = input('Enter url')
img = imread(url)
img_resized=resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data=np.array(flat_data)
print(img.shape)
y_out=model.predict(flat_data)
y_out=categories[y_out[0]]
print(f'PREDICTED OUTPUT: {y_out}')

Enter urlhttps://www.lima-europe.eu/wp-content/uploads/2017/02/shutterstock_102675152.jpg
(4134, 4134, 3)
PREDICTED OUTPUT: Tomato
time: 1min 6s (started: 2021-05-19 11:13:49 +00:00)


In [13]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

     |████████████████████████████████| 8.2MB 2.7MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
     |████████████████████████████████| 112kB 45.3MB/s 
     |████████████████████████████████| 4.2MB 37.2MB/s 
     |████████████████████████████████| 174kB 44.6MB/s 
     |████████████████████████████████| 122kB 45.0MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
time: 19.1 s (started: 2021-05-19 11:14:56 +00:00)


In [14]:
%%writefile App.py
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
st.title('Image Classifisication  using machine learning')
st.text('Upload the image ')
model=pickle.load(open('img_model.p','rb'))
uploaded_file=st.file_uploader("Choose an image....",type='jpg')
if uploaded_file is not None:
  img=Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')
  if st.button('PREDICT'):
    categories = ['Carrot','Spinach','Tomato']
    st.write('Result.....')
    flat_data=[]
    img =np.array(img)
    img_resized=resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data=np.array(flat_data)
    y_out=model.predict(flat_data)
    y_out=categories[y_out[0]]
    st.title(f'PREDICTED OUTPUT: {y_out}')

Writing App.py
time: 4.94 ms (started: 2021-05-19 11:15:15 +00:00)


In [15]:
!nohup streamlit run App.py &
url=ngrok.connect(port='8501')
url


nohup: appending output to 'nohup.out'


'http://a095d607976d.ngrok.io'

time: 1.69 s (started: 2021-05-19 11:15:15 +00:00)
